In [2]:
import numpy as np
import gym
import random
import gym_village2

In [3]:
ENV_NAME = "village2-v0"
env = gym.make(ENV_NAME)
env.render()

Week: 3. Planted Corn: 0, Harvested Corn: 1, Money: 45, LaborDays: 16


In [4]:
print("Number of actions: %d" % env.action_space.n)
print("Number of states: %d" % env.observation_space.n)

Number of actions: 4
Number of states: 222768


In [5]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [6]:
np.random.seed(123)
env.seed(123)

[123]

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape
from keras.optimizers import Adam

In [9]:
env.reset()
env.step(env.action_space.sample())[0]

22380

In [10]:
model_only_embedding = Sequential()
model_only_embedding.add(Embedding(222768, 4, input_length=1))
model_only_embedding.add(Reshape((4,)))
print(model_only_embedding.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 4)              891072    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4)                 0         
Total params: 891,072
Trainable params: 891,072
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model = Sequential()
model.add(Embedding(222768, 10, input_length=1))
model.add(Reshape((10,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(action_size, activation='linear'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 10)             2227680   
_________________________________________________________________
reshape_2 (Reshape)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                550       
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 204       
Total params: 2,233,534
Trainable params: 2,233,534
Non-trainable params: 0
____________________________________________

In [12]:
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
import tensorflow as tf

memory = SequentialMemory(limit=500000, window_length=1)
policy = EpsGreedyQPolicy()
dqn_only_embedding = DQNAgent(model=model, nb_actions=action_size, memory=memory, nb_steps_warmup=500, target_model_update=1e-2, policy=policy)
dqn_only_embedding.compile(Adam(lr=1e-3), metrics=['mae'])


In [14]:
#tf.compat.v1.disable_eager_execution()
dqn_only_embedding.fit(env, nb_steps=2000, visualize=False, verbose=1, nb_max_episode_steps=99, log_interval=500)

Training for 2000 steps ...
Interval 1 (0 steps performed)
500/500 [==============================] - 1s 1ms/step - reward: 0.5740
43 episodes - episode_reward: 6.674 [-1.000, 10.000] - prob: 1.000

Interval 2 (500 steps performed)
500/500 [==============================] - 31s 61ms/step - reward: 0.7500
43 episodes - episode_reward: 8.721 [-1.000, 10.000] - loss: 0.042 - mae: 3.107 - mean_q: 4.782 - prob: 1.000

Interval 3 (1000 steps performed)
500/500 [==============================] - 32s 63ms/step - reward: 0.7060
43 episodes - episode_reward: 8.209 [-1.000, 10.000] - loss: 0.003 - mae: 4.127 - mean_q: 6.096 - prob: 1.000

Interval 4 (1500 steps performed)
500/500 [==============================] - 32s 64ms/step - reward: 0.6400
done, took 94.733 seconds


In [15]:
dqn_only_embedding.test(env, nb_episodes=20, visualize=True, nb_max_episode_steps=99)

Testing for 20 episodes ...
  (Plant Corn)
Week: 1. Planted Corn: 1, Harvested Corn: 0, Money: 45, LaborDays: 15
  (Plant Corn)
Week: 2. Planted Corn: 2, Harvested Corn: 0, Money: 40, LaborDays: 10
  (Harvest Corn)
Week: 3. Planted Corn: 2, Harvested Corn: 0, Money: 35, LaborDays: 10
  (Plant Corn)
Week: 4. Planted Corn: 2, Harvested Corn: 0, Money: 30, LaborDays: 10
  (Plant Corn)
Week: 5. Planted Corn: 2, Harvested Corn: 0, Money: 25, LaborDays: 10
  (Harvest Corn)
Week: 6. Planted Corn: 1, Harvested Corn: 1, Money: 20, LaborDays: 9
  (Harvest Corn)
Week: 7. Planted Corn: 0, Harvested Corn: 2, Money: 15, LaborDays: 8
  (Plant Corn)
Week: 8. Planted Corn: 0, Harvested Corn: 2, Money: 10, LaborDays: 8
  (Plant Corn)
Week: 9. Planted Corn: 0, Harvested Corn: 2, Money: 5, LaborDays: 8
  (Sell Corn)
Week: 10. Planted Corn: 0, Harvested Corn: 1, Money: 20, LaborDays: 8
  (Plant Corn)
Week: 11. Planted Corn: 0, Harvested Corn: 1, Money: 15, LaborDays: 8
  (Sell Corn)
Week: 12. Planted Corn:

Week: 11. Planted Corn: 0, Harvested Corn: 1, Money: 15, LaborDays: 8
  (Sell Corn)
Week: 12. Planted Corn: 0, Harvested Corn: 1, Money: 10, LaborDays: 8
Episode 20: reward: 10.000, steps: 12
